# A simple function

![](img/Fibonacci_staircase_at_St_Johns_College_Cambridge_UK_Valerian_Guillot.jpg)

In [ ]:
%%file fib.py
def fib(n):
    if n <= 1: return n
    else: return fib(n - 2) + fib(n - 1)

## ...can be tested interactively

In [ ]:
%reload_ext autoreload
%autoreload 2
import fib
fib.fib(5)

## Looks about right, ship it.

# But it could `assert` itself...

In [ ]:
%%file fib.py
def fib(n):
    """ Return the `n`th fibonacci sequence number
    >>> assert fib(1) == 1
    >>> assert fib(5) == 5
    """
    if n <= 1: return n
    else: return fib(n - 2) + fib(n - 1)

In [ ]:
import fib
print(fib.fib(1))
print(fib.fib(10))

## ... and be **unit** tested with `doctest`


In [ ]:
%%file test_fib.py
import fib
__import__("doctest").testmod(fib)

In [ ]:
!python3 test_fib.py -v

# What might we do with a **unit** test suite?

## measure **coverage** with `coverage`

In [ ]:
!coverage run --branch --source fib test_fib.py
!coverage report

## infer **type** stubs with `monkeytype`

In [ ]:
!monkeytype run test_fib.py
!monkeytype stub fib

## or apply **type hints** in the source

In [ ]:
!monkeytype apply fib

# **type hints** add intent

In [ ]:
from pathlib import Path; from IPython.display import Code
fib_path = Path("fib.py"); Code(fib_path)

## ... but only do so much alone

In [ ]:
try:
    fib.fib("foo")  # actually have to enter the code
except Exception as err:
    print(err)

# **type check** with `beartype`

In [ ]:
fibear_path = Path("fibear.py")
fibear_path.write_text(f"""
from beartype import beartype
@beartype
{fib_path.read_text().strip() }
""")

In [ ]:
import fibear
try:
    fibear.fib("bear")  # rejects bad inputs at the door
except Exception as err:
    print(err)

## ...but what does it **cost**?

# **benchmark** with `timeit`

In [ ]:
t0 = %timeit -o fib.fib(10)

In [ ]:
t1 = %timeit -o fibear.fib(10)

## ... are these **type hints** really worth it?

# **compile** with `mypyc`

In [ ]:
%reload_ext mypyc_ipython

In [ ]:
%%mypyc
def fib_c(n: int) -> int:
    if n <= 1: return n
    else: return fib_c(n - 2) + fib_c(n - 1)

In [ ]:
t3 = %timeit -o fib_c(10)

In [ ]:
[f"{int(t.average / t3.average)}x" for t in [t0, t1]]

# **profile** with `snakeviz` 

In [ ]:
%reload_ext snakeviz
%snakeviz fibear.fib(20); fib.fib(20); fib_c(20)

# **fuzz** test with `atheris`

In [ ]:
%%file fuzz.py
import atheris

with atheris.instrument_imports():
    import fib, fibear, sys

def TestOneInput(data):
    fdp = atheris.FuzzedDataProvider(data)
    n = fdp.ConsumeInt(2)
    fib.fib(n)
    fibear.fib(n)

atheris.Setup(sys.argv, TestOneInput)
atheris.Fuzz()

In [ ]:
!coverage run --branch --source=fib,fibear fuzz.py -atheris_runs=3
!coverage report

# Up the stack

[A simple web app](./02-app.ipynb)